# Imports

In [1]:
from eeg import EEG
from MyGAN import *
from gan_input_functions import *
from helper_functions import *


import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms, utils
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt
# torch.manual_seed(0) # Set for our testing purposes, please do not change!

import pandas as pd
import numpy as np

2023-10-16 23:17:45.488035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 23:17:45.488087: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 23:17:45.488125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-16 23:17:45.497526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow not install, you could not use those pipelines


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Getting data

In [2]:
eeg_data = EEG(subject_id=[1])

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain desi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.

...

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 165 samples (1.650 s)

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB sto

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


# Configuration

In [3]:
PATH = "../Weights/Conv_Cond_Gan_v1/Gen_Weights.pt"

In [4]:
gen = Generator()

In [5]:
gen.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## Generate Samples

In [6]:
n_epochs = 500
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00002
n_classes = 4
device = 'cuda'


def generate_samples_with_labels(label, n_samples, generator, z_dim = 64, channel = None, extra_dim = True):
    '''
    Function for generating samples, once the generator has been trained
        label: label of the movement to be sampled. See dictionary below
        {'feet': 0, 'left_hand': 1, 'right_hand': 2, 'tongue': 3}
        n_samples: number of samples to be generated
        channel: electrode {'C3': 0, 'Cz': 1, 'C4': 2} -> Default: All channels
        generator: the trained generator
    '''

    n_classes = 4

    if channel == None:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)

        if extra_dim == False:
            fake = fake.reshape((fake.shape[0], fake.shape[2], fake.shape[3]))
        return fake
    else:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)
        filtered_channel_fake = torch.select(fake, dim = 2, index = channel)

        if extra_dim == False:
            filtered_channel_fake = filtered_channel_fake.reshape((filtered_channel_fake.shape[0], filtered_channel_fake.shape[2]))

        return filtered_channel_fake



## Filter Label and channels of real data

In [7]:
def filter_label_and_channel(eeg_data, label, channel):
    '''
    Function to filter label and channel of original eeg data.
        eeg_data: raw eeg data
        label: class of movement
        channel: electrode --   {'C3': 0, 'Cz': 1, 'C4': 2}
    '''


    mask = torch.where(eeg_data[:][1] == label, 1, 0)
    filtered_eeg = eeg_data[:][0][torch.nonzero(mask).flatten()]
    filtered_eeg = filtered_eeg.reshape((filtered_eeg.shape[0], filtered_eeg.shape[2], filtered_eeg.shape[3] ))
    filtered_channel_eeg = torch.select(filtered_eeg, dim = 1, index = channel)

    return filtered_channel_eeg

## Classification Helper Functions

In [8]:
def generate_samples_for_classification(n_samples, generator, z_dim = 64):
    '''
    Function for generating equal label samples for the classifier.
        n_samples: number of samples to be generated
        generator: the trained generator
    '''
    n_classes = 4

    n_samples_partial = int(n_samples/n_classes)
    noise_4_gen = get_noise(n_samples, z_dim)

    label = [0,1,2,3]

    label = [get_one_hot_labels(torch.Tensor([i]).long(), n_classes).repeat(n_samples_partial,1) for i in label]

    label_concat = torch.zeros_like(torch.Tensor(0,4))
    for i in range(len(label)):
        label_concat = torch.cat((label_concat,label[i]), 0)

    noise_and_labels = combine_vectors(noise_4_gen, label_concat)

    fake = generator(noise_and_labels)

    original_labels = torch.argmax(label_concat,dim = 1)
    return (fake,original_labels)


In [9]:
def add_real_fake(real_eeg, fake_eeg):

    real_data = real_eeg[:]
    fake_data = fake_eeg

    complete_eeg_data = torch.cat((real_data[0], fake_data[0]), dim = 0)
    complete_label_data = torch.cat((real_data[1], fake_data[1]), dim = 0)

    return (complete_eeg_data, complete_label_data)


## Classifier

In [10]:
eeg_data[:][0].shape

torch.Size([1152, 1, 3, 400])

In [11]:
from math import floor
def compute_dim_Conv2D(Hin, 
                kernel_size=3, 
                stride=1,
                padding = 0,
                dilation = 1
                ):
    return floor( (Hin + (2*padding) - (dilation*(kernel_size-1)) -1)/(stride) + 1 ) 

def compute_dim_TransConv2D(Hin, 
                        kernel_size=118, 
                        stride=1,
                        padding = 0,
                        dilation = 1,
                        output_padding = 0
        
        ):
    return ((Hin - 1)*stride) - (2*padding)+ (dilation*(kernel_size - 1)) + output_padding + 1


def compute_maxpool_2d(Hin, 
                        kernel_size=4, 
                        padding = 0,
                        dilation = 1        
        ):

    stride = kernel_size
    return ( ( (Hin + (2*padding)-(dilation*(kernel_size - 1)) -1) / stride ) + 1)
 

compute_dim_Conv2D(36)

34

In [12]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.T = 400

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 400), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.tconv2d1 = nn.ConvTranspose2d(16, 16, kernel_size = (118,1), stride = (1,1))
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 2))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 1)

        # # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d(2, 1)

        # # # FC Layer
        # # # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # # # I have 120 timepoints.
        self.fc1 = nn.Linear(4*14*33, 4)


    def forward(self, x):

        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = self.conv2(x)
        x = F.elu(x)
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.reshape(-1, 4*14*33)
        x = F.softmax(self.fc1(x), dim = 1)
        
        return x

In [23]:
## data
X_train_eeg = eeg_data[:][0].to(torch.float32)
y_train_eeg = eeg_data[:][1].to(torch.float32)

In [14]:
classif = Classifier().cuda(0)
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classif.parameters())

# input = Variable(X_train_eeg[0:10]).cuda(0)

In [15]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100

    predicted = []

    for i in range(int(len(X)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size

        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)

        predicted.append(pred.data.cpu().numpy())


    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)

    predicted = predicted.data.cpu().numpy()

    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [16]:
# X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
# y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

# X_val = np.random.rand(100, 1, 120, 64).astype('float32')
# y_val = np.round(np.random.rand(100).astype('float32'))

# X_test = np.random.rand(100, 1, 120, 64).astype('float32')
# y_test = np.round(np.random.rand(100).astype('float32'))

In [25]:
batch_size = 32
epochs = 10
for epoch in range(epochs):  # loop over the dataset multiple times
    # print ("\nEpoch ", epoch)

    running_loss = 0.0
    for i in range(int(len(X_train_eeg)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train_eeg[s:e].numpy())
        labels = torch.FloatTensor(np.array([y_train_eeg[s:e].numpy()]).T*1.0)
        # labels = torch.Tensor(labels.flatten())
        
        # # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # # zero the parameter gradients
        optimizer.zero_grad()
        
        outputs = classif(inputs)
        outputs = torch.argmax(outputs, dim = 1).to(torch.float32)
        outputs = outputs.reshape((-1,1))
        
        # loss = criterion(outputs, labels)
        # loss.backward()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)

    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size

        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)

        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()


        optimizer.step()

        running_loss += loss.item()

    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, X_train, y_train, params))
    print ("Validation - ", evaluate(net, X_val, y_val, params))
    print ("Test - ", evaluate(net, X_test, y_test, params))